In [1]:
!pip install adaptive_classifier

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
import torch.cuda
from adaptive_classifier import AdaptiveClassifier
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

In [4]:
train_df = pd.read_csv("./data/training.csv")
test_df = pd.read_csv("./data/test.csv")

X_train = train_df["text"].tolist()
y_train = train_df["label"].tolist()

X_test = test_df["text"].tolist()
y_test = test_df["label"].tolist()

In [5]:
print(f"Cuda is {'available' if torch.cuda.is_available() else 'unavailable'}")

clf = AdaptiveClassifier(
    "bert-base-uncased",
    "cuda"
)

Cuda is available


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
print("Read data, starting training")

batch_size = 256
for i in tqdm(range(0, len(X_train), batch_size)):
    clf.add_examples(X_train[i:i+batch_size], y_train[i:i+batch_size])

clf.save("./clf")

print("Training done")

Read data, starting training


100%|██████████| 63/63 [08:20<00:00,  7.94s/it]


Training done


In [20]:
predictions = clf.predict_batch(X_test)
print(prediction)

[[(5, 0.39943121556238953), (1, 0.1451999508419665), (4, 0.14088294018910738), (0, 0.13850631093452742), (3, 0.13850631093452742)], [(5, 0.4374845721058975), (1, 0.1428133082705349), (4, 0.14132742853855954), (2, 0.13999606379117804), (0, 0.13837862729381722)], [(4, 0.3267604996013154), (1, 0.14051410766219083), (2, 0.1397195542824125), (5, 0.13971883118729256), (0, 0.1382455320364917)], [(2, 0.4372712586579635), (1, 0.14341295482105565), (4, 0.14049650579073222), (3, 0.13940461955820688), (0, 0.13938461508113661)], [(1, 0.29940492095290266), (3, 0.14108078167474616), (0, 0.14102996293754802), (5, 0.14002062195862602), (2, 0.1399032148157322)], [(3, 0.4388471047348669), (1, 0.14260317856779423), (4, 0.14155980292662668), (0, 0.13884757622065294), (2, 0.13814233755005928)], [(3, 0.43987730600954505), (5, 0.14039633270467078), (2, 0.14039633270456037), (0, 0.13987941290745415), (4, 0.13945061567376973)], [(5, 0.29968592982866427), (1, 0.14288854201080153), (4, 0.14071793543648012), (3, 0

In [22]:
top_predictions = [prediction[0][0] for prediction in predictions]

[5, 5, 4, 2, 1, 3, 3, 5, 1, 5, 1, 2, 4, 5, 2, 5, 1, 3, 2, 2, 2, 5, 2, 4, 3, 4, 3, 3, 2, 2, 2, 5, 0, 5, 3, 1, 2, 0, 2, 5, 2, 2, 2, 4, 5, 5, 2, 2, 5, 5, 2, 4, 4, 0, 5, 2, 3, 4, 2, 2, 0, 0, 5, 5, 0, 5, 4, 1, 5, 4, 5, 4, 5, 2, 5, 2, 2, 4, 2, 2, 1, 3, 5, 4, 5, 5, 2, 2, 1, 0, 5, 3, 1, 3, 4, 2, 2, 4, 0, 2, 2, 5, 3, 3, 2, 3, 5, 5, 3, 5, 2, 2, 4, 5, 3, 2, 1, 5, 0, 5, 0, 2, 3, 2, 4, 2, 0, 3, 3, 2, 4, 2, 4, 3, 2, 2, 4, 4, 2, 5, 2, 4, 1, 5, 2, 3, 5, 5, 5, 2, 5, 2, 5, 3, 5, 4, 5, 2, 0, 2, 3, 5, 1, 4, 5, 2, 5, 2, 2, 2, 2, 5, 2, 5, 0, 2, 2, 2, 3, 5, 5, 2, 5, 1, 4, 5, 2, 1, 5, 2, 1, 2, 2, 2, 2, 5, 4, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 0, 3, 2, 3, 3, 5, 5, 5, 3, 2, 2, 4, 2, 5, 3, 5, 5, 4, 5, 3, 2, 5, 5, 0, 0, 5, 1, 5, 1, 2, 1, 4, 2, 4, 5, 2, 3, 2, 0, 5, 2, 1, 4, 2, 1, 0, 3, 2, 3, 1, 2, 0, 3, 4, 2, 2, 2, 2, 2, 2, 1, 5, 2, 5, 3, 5, 0, 2, 5, 1, 0, 5, 4, 5, 2, 0, 1, 5, 5, 2, 0, 5, 5, 2, 3, 4, 4, 2, 4, 3, 2, 5, 4, 3, 5, 5, 2, 5, 4, 4, 2, 1, 0, 1, 2, 4, 5, 2, 5, 2, 5, 3, 5, 5, 3, 5, 2, 0, 2, 0, 2, 0, 4, 4, 5, 0, 

In [24]:
print(top_predictions)
print(y_test)

[5, 5, 4, 2, 1, 3, 3, 5, 1, 5, 1, 2, 4, 5, 2, 5, 1, 3, 2, 2, 2, 5, 2, 4, 3, 4, 3, 3, 2, 2, 2, 5, 0, 5, 3, 1, 2, 0, 2, 5, 2, 2, 2, 4, 5, 5, 2, 2, 5, 5, 2, 4, 4, 0, 5, 2, 3, 4, 2, 2, 0, 0, 5, 5, 0, 5, 4, 1, 5, 4, 5, 4, 5, 2, 5, 2, 2, 4, 2, 2, 1, 3, 5, 4, 5, 5, 2, 2, 1, 0, 5, 3, 1, 3, 4, 2, 2, 4, 0, 2, 2, 5, 3, 3, 2, 3, 5, 5, 3, 5, 2, 2, 4, 5, 3, 2, 1, 5, 0, 5, 0, 2, 3, 2, 4, 2, 0, 3, 3, 2, 4, 2, 4, 3, 2, 2, 4, 4, 2, 5, 2, 4, 1, 5, 2, 3, 5, 5, 5, 2, 5, 2, 5, 3, 5, 4, 5, 2, 0, 2, 3, 5, 1, 4, 5, 2, 5, 2, 2, 2, 2, 5, 2, 5, 0, 2, 2, 2, 3, 5, 5, 2, 5, 1, 4, 5, 2, 1, 5, 2, 1, 2, 2, 2, 2, 5, 4, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 0, 3, 2, 3, 3, 5, 5, 5, 3, 2, 2, 4, 2, 5, 3, 5, 5, 4, 5, 3, 2, 5, 5, 0, 0, 5, 1, 5, 1, 2, 1, 4, 2, 4, 5, 2, 3, 2, 0, 5, 2, 1, 4, 2, 1, 0, 3, 2, 3, 1, 2, 0, 3, 4, 2, 2, 2, 2, 2, 2, 1, 5, 2, 5, 3, 5, 0, 2, 5, 1, 0, 5, 4, 5, 2, 0, 1, 5, 5, 2, 0, 5, 5, 2, 3, 4, 4, 2, 4, 3, 2, 5, 4, 3, 5, 5, 2, 5, 4, 4, 2, 1, 0, 1, 2, 4, 5, 2, 5, 2, 5, 3, 5, 5, 3, 5, 2, 0, 2, 0, 2, 0, 4, 4, 5, 0, 

In [26]:
print(f"Accuracy: {accuracy_score(y_test, top_predictions)}")
print("Classification Report:\n", classification_report(y_test, top_predictions))

Accuracy: 0.272
Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.17      0.27       581
           1       0.77      0.19      0.31       695
           2       0.14      0.69      0.24       159
           3       0.41      0.31      0.35       275
           4       0.34      0.34      0.34       224
           5       0.08      0.62      0.15        66

    accuracy                           0.27      2000
   macro avg       0.41      0.39      0.28      2000
weighted avg       0.58      0.27      0.30      2000

